# Preamble

This tutorial depends on the NumPy, SciPy, matplotlib, intervaltree, and scikit-learn packages.

In [ ]:
import numpy as np                                       # fast vectors and matrices
import matplotlib.pyplot as plt                          # plotting
from scipy import fft                                    # fast fourier transform

from IPython.display import Audio

from intervaltree import Interval,IntervalTree

from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score

%matplotlib inline

### Constants

In [ ]:
fs = 44100            # samples/second
window_size = 2048    # fourier window size
d = 1024              # number of features
m = 128               # number of distinct notes
stride = 512          # samples between windows
wps = fs/float(512)   # windows/second
n = 1000              # training data points per recording

### Load MusicNet

Download MusicNet from http://homes.cs.washington.edu/~thickstn/musicnet.html. See the Introductory tutorial for more information about the dataset.

In [ ]:
data = np.load(open('musicnet.npz','rb'))

# split our dataset into train and test
test_data = ['2303','2382','1819']
train_data = [f for f in data.files if f not in test_data]

In [ ]:
# create the test set
Xtest = np.empty([3*7500,d])
Ytest = np.zeros([3*7500,m])
for i in range(len(test_data)):
    X,Y = data[test_data[i]]
    for j in range(7500):
        s = fs+j*512 # start from one second to give us some wiggle room for larger segments
        Xtest[7500*i + j] = np.abs(fft(X[s:s+window_size]))[0:d]
        
        # label stuff that's on in the center of the window
        for label in Y[s+d/2]:
            Ytest[7500*i + j,label.data[1]] = 1

# Linear Model

We will learn a linear model using spectrograms as features. See the Spectrograms tutorial for more information about this popular audio featurization. We will learn this model with least squares. The dataset is too big to fit every datapoint, so we will sample the dataset and fit to the sampled subset. Even the sampled subset may be too big to fit in memory, so we will stream it and compute sufficient statistics as we go.

In [ ]:
# sufficient statistics for least squares
XTX = np.zeros((d,d))
XTY = np.zeros((d,m))

# Warning: this could take some time
Xs = np.empty((n,d))
for recording in train_data:
    print recording, ',',
    X,Y = data[recording]
    s = np.random.randint(window_size/2,len(X)-window_size/2,n)
    Ys = np.zeros((n,m))
    for i in range(n):
        Xs[i] = np.abs(fft(X[s[i]-window_size/2:s[i]+window_size/2]))[0:d]
        for label in Y[s[i]]:
            Ys[i,label.data[1]] = 1
    XTX += (1./n)*np.dot(Xs.T,Xs)
    XTY += (1./n)*np.dot(Xs.T,Ys)
XTX /= float(len(train_data))
XTY /= float(len(train_data))

Using the sufficient statistics computed above, we can solve the linear system $X^TXw = X^TY$. We search over a log-scale grid for an appropriate regularizer.

In [ ]:
grid = [2**i for i in range(-5,5)]
average_precision = []
for r in grid:
    print r,', ',
    w = np.linalg.solve(XTX + r*np.eye(XTX.shape[0]),XTY)
    
    Yhat = np.dot(Xtest,w)
    yflat = Ytest.reshape(Ytest.shape[0]*Ytest.shape[1])
    yhatflat = Yhat.reshape(Yhat.shape[0]*Yhat.shape[1])
    average_precision.append(average_precision_score(yflat, yhatflat))
    
fig = plt.figure()
plt.plot(range(-5,5),average_precision,color=(41/255.,104/255.,168/255.),linewidth=3)
fig.axes[0].set_xlabel('regulizer (order of magnitude)')
fig.axes[0].set_ylabel('average precision')

We can plot a complete precision-recall curve for a particular estimator.

In [ ]:
w = np.linalg.solve(XTX + 2*np.eye(XTX.shape[0]),XTY)
Yhat = np.dot(Xtest,w)
yflat = Ytest.reshape(Ytest.shape[0]*Ytest.shape[1])
yhatflat = Yhat.reshape(Yhat.shape[0]*Yhat.shape[1])
precision, recall, _ = precision_recall_curve(yflat, yhatflat)

fig = plt.figure()
plt.plot(recall,precision,color=(41/255.,104/255.,168/255.),linewidth=3)
fig.axes[0].set_xlabel('recall')
fig.axes[0].set_ylabel('precision')

And we can break our analysis down into subsets of the test set, considering data points with particular numbers of labels separately.

In [ ]:
average_precision.append(average_precision_score(yflat, yhatflat))

Ytest1 = Ytest[np.sum(Ytest,axis=1)==1]
yflat = Ytest1.reshape(Ytest1.shape[0]*Ytest1.shape[1])
Yhat = np.dot(Xtest[np.sum(Ytest,axis=1)==1],w)
yhatflat = Yhat.reshape(Yhat.shape[0]*Yhat.shape[1])
precision1, recall1, _ = precision_recall_curve(yflat, yhatflat)

Ytest3 = Ytest[np.sum(Ytest,axis=1)==3]
yflat = Ytest3.reshape(Ytest3.shape[0]*Ytest3.shape[1])
Yhat = np.dot(Xtest[np.sum(Ytest,axis=1)==3],w)
yhatflat = Yhat.reshape(Yhat.shape[0]*Yhat.shape[1])
precision3, recall3, _ = precision_recall_curve(yflat, yhatflat)

fig = plt.figure()
total, = plt.plot(recall,precision,color=(41/255.,104/255.,168/255.),linewidth=3)
one, = plt.plot(recall1,precision1,color=(70/255.,179/255.,76/255.),linewidth=3)
three, = plt.plot(recall3,precision3,color=(180/255.,50/255.,47/255.),linewidth=3)
ax = fig.axes[0]
leg = ax.legend([total,one,three],['overall','one-note','three-notes'],\
          loc='upper right',ncol=1,prop={'size':13})
for legobj in leg.legendHandles:
    legobj.set_linewidth(7.0)
ax.set_xlabel('recall')
ax.set_ylabel('precision')
plt.savefig('prcurve.eps',format='eps', dpi=1000)